In [27]:
from calendar import monthrange
from datetime import datetime
import pandas as pd
from pandas.api.types import CategoricalDtype
import os
import json

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
sys.path.append(os.path.abspath("../../tools"))
from edahelpers import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [2]:
os.path.abspath("../../tools")

'C:\\Users\\mchau\\Documents\\co2-flux-hourly-gpp-modeling\\code\\src\\tools'

In [3]:
root_dir =  os.path.abspath(os.getcwd()+"..\\..\\..\\..\\..\\")
tmp_dir = root_dir + "\\.tmp"
data_dir = root_dir + "\\data\\"

site_metadata_filename = data_dir + "site-metadata.csv"

In [4]:
# "Golden" Sites
target_site = ["US-MMS", "US-Vcp", "FR-Pue", "CH-Lae", "US-Var", "US-Ne2", "ES-LJu", "US-Ton"]

In [5]:
# Load Site data
site_metadata_df = pd.read_csv(site_metadata_filename)

# only focus on target sites
site_metadata_df= site_metadata_df.loc[site_metadata_df['site_id'].isin(target_site)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df

size:(8, 26)


,site_id,dataset,start_year,end_year,file,is_dup,IGBP,elevation,lat,long,site_name,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent,filename,size,country,record_count,site_IGBP,site_koppen,start_time,end_time,recorded_day_count
67,FR-Pue,FLUXNET,2000,2014,FLX_FR-Pue_FLUXNET2015_FULLSET_MM_2000-2014_2-...,False,EBF,270.0,43.74130,3.59570,Puechabon,8,3,Csa,Temperate,C3,6.59,data_full_half_hourly_raw_v0_1_FR-Pue.csv,109116169.0,FR,245760.0,EBF,Temperate,2000-07-26 00:00:00,2014-12-31 23:30:00,5120.0
119,US-Ne2,FLUXNET,2001,2013,FLX_US-Ne2_FLUXNET2015_FULLSET_MM_2001-2013_1-...,False,CRO,362.0,41.16487,-96.47010,Mead - irrigated maize-soybean rotation site,25,4,Dfa,Cold,rotation,48.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,US-Ton,FLUXNET,2001,2014,FLX_US-Ton_FLUXNET2015_FULLSET_MM_2001-2014_1-...,False,WSA,177.0,38.43160,-120.96598,Tonzi Ranch,8,3,Csa,Temperate,C3,0.00,data_full_half_hourly_raw_v0_1_US-Ton.csv,103999932.0,US,230928.0,WSA,Temperate,2001-05-24 00:00:00,2014-12-31 23:30:00,4811.0
130,US-Var,FLUXNET,2000,2014,FLX_US-Var_FLUXNET2015_FULLSET_MM_2000-2014_1-...,False,GRA,129.0,38.41330,-120.95070,Vaira Ranch- Ione,8,3,Csa,Temperate,C3,0.00,data_full_half_hourly_raw_v0_1_US-Var.csv,110098318.0,US,245712.0,GRA,Temperate,2000-11-01 00:00:00,2014-12-31 23:30:00,5119.0
181,US-MMS,AmeriFlux,1999,2017,FLX_US-MMS_FLUXNET2015_FULLSET_MM_1999-2017_be...,True,DBF,275.0,39.32320,-86.41310,Morgan Monroe State Forest,25,4,Dfa,Cold,C3,42.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,US-Vcp,AmeriFlux,2007,2017,FLX_US-Vcp_FLUXNET2015_FULLSET_MM_2007-2017_be...,False,ENF,2542.0,35.86240,-106.59740,Valles Caldera Ponderosa Pine,26,4,Dfb,Cold,C3,0.04,data_full_half_hourly_raw_v0_1_US-Vcp.csv,72934242.0,US,174528.0,ENF,Cold,2007-01-01 00:00:00,2017-12-31 23:30:00,3636.0
219,CH-Lae,ICOS2020,2004,2020,FLX_CH-Lae_FLUXNET2015_FULLSET_MM_2004-2020_be...,True,MF,689.0,47.47833,8.36439,Laegern,26,4,Dfb,Cold,C3,0.00,data_full_half_hourly_raw_v0_1_CH-Lae.csv,116210397.0,CH,288384.0,MF,Cold,2004-04-09 00:00:00,2020-12-17 23:30:00,6008.0
244,ES-LJu,ICOS2020,2004,2020,FLX_ES-LJu_FLUXNET2015_FULLSET_MM_2004-2020_be...,True,OSH,1600.0,36.92659,-2.75212,Llano de los Juanes,8,3,Csa,Temperate,C3,0.00,data_full_half_hourly_raw_v0_1_ES-LJu.csv,111661016.0,ES,239616.0,OSH,Temperate,2004-05-26 00:00:00,2020-12-31 23:30:00,4992.0


# Define Features

In [ ]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']
qc_flag_dtype = CategoricalDtype([0, 1, 2, 3], ordered=True)
qc_flags_features = [s for s in all_features if "_QC" in s]
qc_flags_features

# Process Single File

In [64]:
r = site_metadata_df[['site_id','filename']].iloc[-1]
print(r.filename)
local_filename = tmp_dir + "\\" + r.filename

data_full_half_hourly_raw_v0_1_ES-LJu.csv


In [65]:
site_df = pd.read_csv(local_filename)

In [66]:
#features = ['SITE_ID','TIMESTAMP_START', 'TIMESTAMP_END', 'datetime', 'date', 'year', 'month', 'day', 'hour',"GPP_NT_VUT_REF"]
#site_df = pd.read_csv(local_filename, usecols=features + qc_glags_features)
site_df['datetime'] = pd.to_datetime(site_df['datetime'])
site_df['date'] = pd.to_datetime(site_df['date'])
site_df[qc_flags_features] = site_df[qc_flags_features].astype(qc_flag_dtype)
print(f"size:{site_df.shape}")
site_df.head()

size:(239616, 64)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200405260000,200405260030,7.856,0,8.325,0.0,0.0,0,0.0,314.215,2,314.215,0.0,0,3.144,0.194,2,0.194,84.186,2,84.186,-32.207,0.0,-17.4639,0,-0.130344,1,-0.181093,-1.57822,1,-2.19269,1.24478,1,0.877808,2,NaN,0.0,-0.224026,0.0,NaN,2.99961,0.670451,1.08157,2004-05-26 00:00:00,2004,5,26,0,ES-LJu,2004-05-26,1.24478,NaN,0.179995,0.326648,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
1,200405260030,200405260100,7.650,0,8.180,0.0,0.0,0,0.0,314.215,2,314.215,0.0,0,3.043,0.194,2,0.194,84.173,2,84.173,-30.418,0.0,-22.0652,0,-0.130344,1,-0.181093,-1.57822,1,-2.19269,1.24478,1,0.877808,2,NaN,0.0,-0.229381,0.0,NaN,2.98894,0.665096,1.05119,2004-05-26 00:30:00,2004,5,26,0,ES-LJu,2004-05-26,1.24478,NaN,0.179995,0.326648,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
2,200405260100,200405260130,7.672,0,8.035,0.0,0.0,0,0.0,328.276,2,328.276,0.0,0,2.942,0.090,2,0.090,84.160,2,84.160,-19.960,0.0,-16.2372,0,-0.130344,1,-0.181093,-1.57822,1,-2.19269,1.24478,1,0.877808,2,NaN,0.0,-0.228809,0.0,NaN,2.99008,0.665668,1.05440,2004-05-26 01:00:00,2004,5,26,1,ES-LJu,2004-05-26,1.24478,NaN,0.179995,0.326648,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
3,200405260130,200405260200,7.793,0,8.300,0.0,0.0,0,0.0,328.276,2,328.276,0.0,0,3.064,0.090,2,0.090,84.137,2,84.137,-25.722,0.0,-14.2753,0,-0.130344,1,-0.181093,-1.57822,1,-2.19269,1.24478,1,0.877808,2,NaN,0.0,-0.225664,0.0,NaN,2.99635,0.668813,1.07221,2004-05-26 01:30:00,2004,5,26,1,ES-LJu,2004-05-26,1.24478,NaN,0.179995,0.326648,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
4,200405260200,200405260230,7.638,0,8.565,0.0,0.0,0,0.0,339.403,2,339.403,0.0,0,3.185,0.047,2,0.047,84.113,2,84.113,-48.710,0.0,-23.5167,0,-0.130344,1,-0.181093,-1.57822,1,-2.19269,1.24478,1,0.877808,2,NaN,0.0,-0.229693,0.0,NaN,2.98832,0.664785,1.04944,2004-05-26 02:00:00,2004,5,26,2,ES-LJu,2004-05-26,1.24478,NaN,0.179995,0.326648,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate


In [67]:
site_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239616 entries, 0 to 239615
Data columns (total 64 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   TIMESTAMP_START  239616 non-null  int64         
 1   TIMESTAMP_END    239616 non-null  int64         
 2   TA_F             239616 non-null  float64       
 3   TA_F_QC          239616 non-null  category      
 4   TA_ERA           239616 non-null  float64       
 5   SW_IN_POT        239616 non-null  float64       
 6   SW_IN_F          239616 non-null  float64       
 7   SW_IN_F_QC       239616 non-null  category      
 8   SW_IN_ERA        239616 non-null  float64       
 9   LW_IN_F          239616 non-null  float64       
 10  LW_IN_F_QC       239616 non-null  category      
 11  LW_IN_ERA        239616 non-null  float64       
 12  VPD_F            239616 non-null  float64       
 13  VPD_F_QC         239616 non-null  category      
 14  VPD_ERA          239

In [68]:
site_df[qc_flags_features].describe()

,TA_F_QC,SW_IN_F_QC,LW_IN_F_QC,VPD_F_QC,P_F_QC,PA_F_QC,G_F_MDS_QC,LE_F_MDS_QC,H_F_MDS_QC,NEE_VUT_REF_QC,NEE_CUT_REF_QC
count,239616,239616,239616,239616,239616,239616,239616,239616,239616,239616,239616
unique,3,3,1,3,2,2,4,4,4,4,4
top,0,0,2,0,0,0,0,0,0,1,1
freq,234618,230126,239616,227728,209448,229143,233686,199073,217577,125237,121605


In [69]:
display(site_df[qc_flags_features].apply(pd.value_counts))

,TA_F_QC,SW_IN_F_QC,LW_IN_F_QC,VPD_F_QC,P_F_QC,PA_F_QC,G_F_MDS_QC,LE_F_MDS_QC,H_F_MDS_QC,NEE_VUT_REF_QC,NEE_CUT_REF_QC
0,234618,230126,0,227728,209448,229143,233686,199073,217577,105053,108549
1,3356,1421,0,7392,0,0,3443,37078,18929,125237,121605
2,1642,8069,239616,4496,30168,10473,1457,2609,2273,7594,7291
3,0,0,0,0,0,0,1030,856,837,1732,2171


# Process All Files

In [73]:
for i, r in site_metadata_df[['site_id','filename']].iterrows():
    if not r.filename or type(r.filename) != type(""):
        print(f'\nERROR: {r.site_id} is mssing hourly data.')
        continue

    local_filename = tmp_dir + "\\" + r.filename
    site_df = pd.read_csv(local_filename)
    site_df['datetime'] = pd.to_datetime(site_df['datetime'])
    site_df['date'] = pd.to_datetime(site_df['date'])
    site_df[qc_flags_features] = site_df[qc_flags_features].astype(qc_flag_dtype)
    
    print(f"\n{r.site_id}")
    if site_df.isna().sum().sum() != 0:
        print(f'{site_df.isna().sum()}\n')
    
    display(site_df[qc_flags_features].apply(pd.value_counts))


FR-Pue
TIMESTAMP_START         0
TIMESTAMP_END           0
TA_F                    0
TA_F_QC                 0
TA_ERA                  0
SW_IN_POT               0
SW_IN_F                 0
SW_IN_F_QC              0
SW_IN_ERA               0
LW_IN_F                 0
LW_IN_F_QC              0
LW_IN_ERA               0
VPD_F                   0
VPD_F_QC                0
VPD_ERA                 0
P_F                     0
P_F_QC                  0
P_ERA                   0
PA_F                    0
PA_F_QC                 0
PA_ERA                  0
NETRAD               4325
PPFD_IN             22234
G_F_MDS            102266
G_F_MDS_QC         102266
LE_F_MDS                0
LE_F_MDS_QC             0
LE_CORR             50149
H_F_MDS                 0
H_F_MDS_QC              0
H_CORR              50149
NEE_VUT_REF             0
NEE_VUT_REF_QC          0
NEE_CUT_REF             0
NEE_CUT_REF_QC          0
GPP_NT_VUT_REF          0
GPP_DT_VUT_REF          0
GPP_NT_CUT_REF          0
GPP_

,TA_F_QC,SW_IN_F_QC,LW_IN_F_QC,VPD_F_QC,P_F_QC,PA_F_QC,G_F_MDS_QC,LE_F_MDS_QC,H_F_MDS_QC,NEE_VUT_REF_QC,NEE_CUT_REF_QC
0,244413,243991,157188,244036,245066,167305,118776,200474,174992,104186,102025
1,656,887,2650,1031,0,0,2590,42644,68291,129287,131020
2,691,882,85922,693,694,78455,5147,2472,2272,11770,12190
3,0,0,0,0,0,0,16981,170,205,517,525



ERROR: US-Ne2 is mssing hourly data.

US-Ton
TIMESTAMP_START         0
TIMESTAMP_END           0
TA_F                    0
TA_F_QC                 0
TA_ERA                  0
SW_IN_POT               0
SW_IN_F                 0
SW_IN_F_QC              0
SW_IN_ERA               0
LW_IN_F                 0
LW_IN_F_QC              0
LW_IN_ERA               0
VPD_F                   0
VPD_F_QC                0
VPD_ERA                 0
P_F                     0
P_F_QC                  0
P_ERA                   0
PA_F                    0
PA_F_QC                 0
PA_ERA                  0
NETRAD               7931
PPFD_IN               233
G_F_MDS                 0
G_F_MDS_QC              0
LE_F_MDS                0
LE_F_MDS_QC             0
LE_CORR                 0
H_F_MDS                 0
H_F_MDS_QC              0
H_CORR                  0
NEE_VUT_REF             0
NEE_VUT_REF_QC          0
NEE_CUT_REF             0
NEE_CUT_REF_QC          0
GPP_NT_VUT_REF          0
GPP_DT_VUT_REF    

,TA_F_QC,SW_IN_F_QC,LW_IN_F_QC,VPD_F_QC,P_F_QC,PA_F_QC,G_F_MDS_QC,LE_F_MDS_QC,H_F_MDS_QC,NEE_VUT_REF_QC,NEE_CUT_REF_QC
0,228390,230048,0,230264,230621,230211,227740,166059,177417,65566,64659
1,2357,695,0,636,0,0,2779,59897,49063,149202,149698
2,181,185,230928,28,307,717,327,4944,4424,15939,16308
3,0,0,0,0,0,0,82,28,24,221,263



US-Var
TIMESTAMP_START         0
TIMESTAMP_END           0
TA_F                    0
TA_F_QC                 0
TA_ERA                  0
SW_IN_POT               0
SW_IN_F                 0
SW_IN_F_QC              0
SW_IN_ERA               0
LW_IN_F                 0
LW_IN_F_QC              0
LW_IN_ERA               0
VPD_F                   0
VPD_F_QC                0
VPD_ERA                 0
P_F                     0
P_F_QC                  0
P_ERA                   0
PA_F                    0
PA_F_QC                 0
PA_ERA                  0
NETRAD              36398
PPFD_IN              1505
G_F_MDS                 0
G_F_MDS_QC              0
LE_F_MDS                0
LE_F_MDS_QC             0
LE_CORR                 0
H_F_MDS                 0
H_F_MDS_QC              0
H_CORR                  0
NEE_VUT_REF             0
NEE_VUT_REF_QC          0
NEE_CUT_REF             0
NEE_CUT_REF_QC          0
GPP_NT_VUT_REF          0
GPP_DT_VUT_REF          0
GPP_NT_CUT_REF          0
GPP_

...

,TA_F_QC,SW_IN_F_QC,LW_IN_F_QC,VPD_F_QC,P_F_QC,PA_F_QC,G_F_MDS_QC,LE_F_MDS_QC,H_F_MDS_QC,NEE_VUT_REF_QC,NEE_CUT_REF_QC
0,164905,171379,0,149022,115066,169772,0,0,0,70948,70404
1,8610,1580,0,9745,0,0,0,0,0,95952,96060
2,1013,1569,174528,15761,59462,4756,0,0,0,6987,7416
3,0,0,0,0,0,0,0,0,0,641,648



CH-Lae
TIMESTAMP_START         0
TIMESTAMP_END           0
TA_F                    0
TA_F_QC                 0
TA_ERA              16176
SW_IN_POT               0
SW_IN_F                 0
SW_IN_F_QC              0
SW_IN_ERA           16176
LW_IN_F                 0
LW_IN_F_QC              0
LW_IN_ERA           16176
VPD_F                   0
VPD_F_QC                0
VPD_ERA             16176
P_F                 16176
P_F_QC              16176
P_ERA               16176
PA_F                16176
PA_F_QC             16176
PA_ERA              16176
NETRAD             288384
PPFD_IN             10368
G_F_MDS            288384
G_F_MDS_QC         288384
LE_F_MDS                0
LE_F_MDS_QC             0
LE_CORR            288384
H_F_MDS                 0
H_F_MDS_QC              0
H_CORR             288384
NEE_VUT_REF             0
NEE_VUT_REF_QC          0
NEE_CUT_REF             0
NEE_CUT_REF_QC          0
GPP_NT_VUT_REF          0
GPP_DT_VUT_REF          0
GPP_NT_CUT_REF          0
GPP_

,TA_F_QC,SW_IN_F_QC,LW_IN_F_QC,VPD_F_QC,P_F_QC,PA_F_QC,G_F_MDS_QC,LE_F_MDS_QC,H_F_MDS_QC,NEE_VUT_REF_QC,NEE_CUT_REF_QC
0,278102,277629,260919,278014,272208,245053,0,182597,226736,89467,94121
1,1258,1173,1497,1346,0,0,0,102424,60632,183615,180453
2,9024,9561,25968,9024,0,27155,0,3087,994,14415,12672
3,0,21,0,0,0,0,0,276,22,887,1138



ES-LJu
TIMESTAMP_START         0
TIMESTAMP_END           0
TA_F                    0
TA_F_QC                 0
TA_ERA                  0
SW_IN_POT               0
SW_IN_F                 0
SW_IN_F_QC              0
SW_IN_ERA               0
LW_IN_F                 0
LW_IN_F_QC              0
LW_IN_ERA               0
VPD_F                   0
VPD_F_QC                0
VPD_ERA                 0
P_F                     0
P_F_QC                  0
P_ERA                   0
PA_F                    0
PA_F_QC                 0
PA_ERA                  0
NETRAD               9976
PPFD_IN             12071
G_F_MDS                 0
G_F_MDS_QC              0
LE_F_MDS                0
LE_F_MDS_QC             0
LE_CORR                 0
H_F_MDS                 0
H_F_MDS_QC              0
H_CORR                  0
NEE_VUT_REF             0
NEE_VUT_REF_QC          0
NEE_CUT_REF             0
NEE_CUT_REF_QC          0
GPP_NT_VUT_REF       8880
GPP_DT_VUT_REF          0
GPP_NT_CUT_REF          0
GPP_

,TA_F_QC,SW_IN_F_QC,LW_IN_F_QC,VPD_F_QC,P_F_QC,PA_F_QC,G_F_MDS_QC,LE_F_MDS_QC,H_F_MDS_QC,NEE_VUT_REF_QC,NEE_CUT_REF_QC
0,234618,230126,0,227728,209448,229143,233686,199073,217577,105053,108549
1,3356,1421,0,7392,0,0,3443,37078,18929,125237,121605
2,1642,8069,239616,4496,30168,10473,1457,2609,2273,7594,7291
3,0,0,0,0,0,0,1030,856,837,1732,2171
